In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import json
import requests

In [36]:
def input_data(driver, class_O, input_date): #driver과 필터값을 주면 해당필터의 목록을 보여주고
                                #값을 받아 해당 카테고리로 이동하는 함수
    
    
    if class_O == "기간":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[1]/dl/dd/ul"""
    elif class_O == "대분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
    elif class_O == "중분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""
    elif class_O == "가격":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[4]/dl/dd/ul"""
    else:
        print("잘못된 input_data함수 사용")
        return 0
    
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.TAG_NAME,"li")


    word = []

    for date in data_list:
        data_a_list = date.find_elements(By.TAG_NAME,"a")
        for data_a in data_a_list:
            word.append(data_a.get_attribute("innerText"))


    
    try:
        if input_date in word:
            num = word.index(input_date)
            a = data_list[num].find_elements(By.TAG_NAME,"a")
            if len(a) >= 2:
                a[1].click()
            else:
                a[0].click()
            #data_list[word.index(input_date)].click()
            
        else:
            raise TypeError 
    except TypeError:
        print("입력가능한 항목이 아닙니다.")
    

def get_product_data(driver):
    product_link = []
    product_names = []
    product_brand = []
    product_gender = []
    product_saleRate = []
    product_goods = []
    product_review_count = []
    product_member_price = []
    product_nomal_price = []
    product_tags = []
    product_bigCategory = []
    product_smallCategory = []
    product_rate = []
    
    xpath = """//*[@id="goodsRankList"]"""
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.CLASS_NAME,"list_img")


    goods_tmp = driver.find_element(By.ID,"goodsRankList")
    goods_like = goods_tmp.find_elements(By.NAME,"count")

    for like in goods_like:
        product_goods.append(like.get_attribute("innerText"))

    for data in data_list:
        link = (data.find_element(By.TAG_NAME,"a").get_attribute("href"))
        product_link.append(link)
        re = Request(url=link, headers={"User-Agent":generate_user_agent()})
        page = urlopen(re)
        soup = BeautifulSoup(page, "html.parser")
        product_names.append(soup.find(id="notexist_title").string)
        
        script_org = soup.find_all("script")
        script = script_org[-1].get_text()#
        script_low = (script[script.find("product.state = {")+len("product.state = {")-1:script.rfind(";")+1])
        val = eval(script_low[:script_low.rfind("}")+1].replace("false","False").replace("null","None").replace("true","True"))
        try:
            product_saleRate.append(val["differenceColorGoods"][0]["saleRate"])
        except IndexError:
            product_saleRate.append(0)
        
        product_rate.append(data_list.index(data))
        product_brand.append(val["brand"])
        product_gender.append(val["sex"])
        product_bigCategory.append(val["category"]["categoryDepth1Title"])
        product_smallCategory.append(val["category"]["categoryDepth2Title"])
        product_review_count.append(val["goodsReview"]["totalCount"])
        product_member_price.append(val["goodsPriceMemberLevels"][0]["price"])
        product_nomal_price.append(val["goodsPrice"]["originPrice"])
        product_tags.append(str(val["goodsTags"]))
    
    df = pd.DataFrame()
    df["링크"] = product_link
    df["이름"] = product_names
    df["브랜드"] = product_brand
    df["성별"] = product_gender
    df["할인율"] = product_saleRate
    df["대분류"] = product_bigCategory
    df["중분류"] = product_smallCategory
    df["좋아요수"] = product_goods
    df["리뷰개수"] = product_review_count
    df["회원가"] = product_member_price
    df["일반가"] = product_nomal_price
    df["태그"] = product_tags
    return df




In [7]:
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/home/rds/amr_ws/edm/data",
        "download.prompt_for_download":False}
options.add_experimental_option("prefs",prefs)
url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
driver = webdriver.Chrome(service=Service("../../driver/chromedriver-linux64/chromedriver"),
                        options=options)
driver.get(url)
driver.maximize_window()
#대기


In [4]:
data_frame = pd.DataFrame()
big_xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
mid_xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""

data_bar = driver.find_element(By.XPATH, big_xpath)
big_list = data_bar.find_elements(By.TAG_NAME,"a")
big_texts = ["아우터", "모자", "가방", "상의", "바지", "신발" ] # 검색 대분류 목록
mid_black_list = ["환절기 코트","겨울 기타 코트", "패딩 베스트",
                   "사파리/헌팅 재킷", "기타 아우터",
                     "레깅스", "힐/펌프스", "플랫 슈즈", "신발 용품"]
# 중분류 블랙리스트


for big_text in big_texts:
    WebDriverWait(driver,10)
    input_data(driver,"대분류",big_text)
    WebDriverWait(driver,10)
    data_bar_mid = driver.find_element(By.XPATH, mid_xpath)
    mid_list = data_bar_mid.find_elements(By.TAG_NAME,"a")
    mid_texts = []
    
    for mid in mid_list:
        if mid.get_attribute("innerText") in mid_black_list:
            continue
        else:
            mid_texts.append(mid.get_attribute("innerText"))

    for mid_text in mid_texts:
        WebDriverWait(driver,10)
        input_data(driver,"중분류",mid_text)
        WebDriverWait(driver,10)
        data_frame = pd.concat([data_frame, get_product_data(driver)], axis=0)
        input_data(driver,"중분류",mid_text)
    input_data(driver,"대분류",big_text)
        






NameError: name 'pd' is not defined

In [18]:
big_text

'아우터'

In [38]:
data_frame

,링크,이름,브랜드,성별,할인율,대분류,중분류,좋아요수,리뷰개수,회원가,일반가,태그
0,https://www.musinsa.com/app/goods/1778404?loc=...,2WAY 스웻 후드 집업 (MELANGE GREY),toffee,"[남성, 여성]",13,아우터,후드 집업,"217,746",81362,38610,45000,"['후드', '집업후드', '트레이닝세트', '셋업', '세트', '후드티', '오..."
1,https://www.musinsa.com/app/goods/3777829?loc=...,LONELY/LOVELY WASHED HOODIE ZIP-UP_SOFT RED,nohant,[여성],0,아우터,후드 집업,34,0,186120,188000,"['집업후드', '오버핏후드집업', '후드집업추천', '오버핏', '후드집업', '..."
2,https://www.musinsa.com/app/goods/3697526?loc=...,[2-WAY] 슬리브 스타 피그먼트 후드 집업 스모크블랙,compagno,[남성],0,아우터,후드 집업,"4,584",97,45446,90000,"['빈티지', '후드집업', '커플후드', '오버핏', '투웨이', '피그먼트']"
3,https://www.musinsa.com/app/goods/2378454?loc=...,MAIN LOGO CROPPED HOOD ZIP-UP HEATHER GRAY,prendafromplant,[여성],70,아우터,후드 집업,"7,233",369,20700,69000,['WEEKLYSALE']
4,https://www.musinsa.com/app/goods/3731139?loc=...,Contrast Logo Zip-Up Hoodie (Brown),aakam,[남성],0,아우터,후드 집업,164,2,149339,151000,"['신상', '23FW', '후드', '후드집업', '피그먼트']"
...,...,...,...,...,...,...,...,...,...,...,...,...
85,https://www.musinsa.com/app/goods/1179323?loc=...,스웨이드 스판 롱 삭스부츠 (BLACK),cesti,[여성],0,신발,부츠,"2,056",106,107280,149000,"['롱부츠', '부츠', '삭스부츠', '여자부츠', '블랙', '스판부츠', '스..."
86,https://www.musinsa.com/app/goods/1184449?loc=...,1391 워시콘 소가죽 첼시 부츠 밴타블랙,miamiproject,"[남성, 여성]",24,신발,부츠,"2,371",490,111955,149000,"['가죽', '마프', '블랙', '수제화', '첼시부츠', '키높이']"
87,https://www.musinsa.com/app/goods/1196016?loc=...,19B515 black ankle,lifla,[여성],61,신발,부츠,775,65,67965,178000,"['부츠', '앵클', '착화감', '편안한', '데일리', '출근룩']"
88,https://www.musinsa.com/app/goods/1196017?loc=...,19B515 suede black ankle,lifla,[여성],61,신발,부츠,"2,092",91,67965,178000,['블랙']


In [17]:
print(product_smallCategory[-1])

기타 모자


In [20]:
data_frame.to_csv(f"../../data/product.csv")

In [62]:
goods_like[0].get_attribute("innerText")

'1,031'

In [34]:
print(product_link[-1])

https://www.musinsa.com/app/goods/1225219?loc=goods_rank


In [37]:
soup2 = BeautifulSoup(page, "html.parser")
soup2.prettify

<bound method Tag.prettify of >

In [36]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="ko">
<head>
<!-- Google Tag Manager -->
<script>
var selfCertEnabled = "" || "Y";
var THIS_PAGE_GF = 'A'; // 전역필터 상태값 caching page reload 필요 체크용
var USE_ORDER_GROUP_SERVICE = "N";
var ORDER_GROUP_STORE_PATH = "/app";
var ORDER_GROUP_ORDER_PATH = "/app";
var dataLayer = window.dataLayer || [];
window.addEventListener("DOMContentLoaded", function() {
    
    (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TXDSFSF');
}, false);
</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://image.msscdn.net/favicon_152.ico" rel="shortcut icon"/>

In [275]:
a = soup.find_all("script")


In [291]:
test = a[21].getText()
test = (test[test.find("{"):test.find(";")])

co = eval(test[:test.rfind("}")+1].replace("false","False").replace("null","None").replace("true","True"))





dict

In [300]:
co["goodsReview"]["totalCountList"][""]

38900

In [225]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="ko">
<head>
<!-- Google Tag Manager -->
<script>
var selfCertEnabled = "" || "Y";
var THIS_PAGE_GF = 'A'; // 전역필터 상태값 caching page reload 필요 체크용
var USE_ORDER_GROUP_SERVICE = "N";
var ORDER_GROUP_STORE_PATH = "/app";
var ORDER_GROUP_ORDER_PATH = "/app";
var dataLayer = window.dataLayer || [];
window.addEventListener("DOMContentLoaded", function() {
    
    (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TXDSFSF');
}, false);
</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://image.msscdn.net/favicon_152.ico" rel="shortcut icon"/>

In [1]:
driver.close()

NameError: name 'driver' is not defined

In [ ]:

#################################################
# 주의 데이터 베이스로 df올리는 블럭임##################
#################################################
import mysql.connector
con = mysql.connector.connect(
    host = "database-1.cd2is2gsweff.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "admin",
    password = "kim82458529",
    database = "amr_base"
)
cur = con.cursor(buffered = True)


sql = "insert into yousinsa values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for i, row in data_frame.iterrows():
    
    cur.execute(sql,tuple(row))
    con.commit()



con.close()

